In [53]:
### Import all the libraries ###
import requests 
import re
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import tensorflow as tf 
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
#We use the wordnet to find the synonym of the word. If the synonym are the same for some words, then there is a high chance that the text is talking about this subject. The synonyms of a word are returned as a nested list of synonyms of the different senses of the input word in the given language, since these different senses are not mutual synonyms#
nltk.download('wordnet')
#We use the omw package with the stopswords#
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import warnings
warnings.filterwarnings('ignore')
import wikipediaapi
################################

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-d

In [58]:
### Create a input to ask wichpage we want to analyse###
name = input('Quel est la page a consulte ?')

Quel est la page a consulte ?Joueur professionnel d'Overwatch


In [80]:
###############################################################################################
#### Scrapp the page of the given category name page to give categories and subcategories #####
##############################################################################################
def get_category(category_name):
    # Using the wikipedia api to give the categories
    wiki_wiki = wikipediaapi.Wikipedia('fr')
    category = wiki_wiki.page('Catégorie:' + category_name)
    pages = []
    if category.exists():
        category_members = category.categorymembers
        for cat_member_title in category_members:
            #Scan all the categories tittle
            if 'Catégorie:' in cat_member_title:
                pages += get_category(cat_member_title[10:])
            else:
                page = wiki_wiki.page(cat_member_title)
                #adding theme to a list
                pages.append(page.title)
    else:
        print("Category does not exist")
    return pages
pages = get_category(name)
print(pages)

['Guerres médiques', 'Calliope (Hérodote)', 'Canal de Xerxès', "Destruction achéménide d'Athènes", 'Guerres de la Ligue de Délos', 'Mardonios', 'Paix de Callias', 'Les Perses', 'Première guerre médique', 'Seconde guerre médique', 'Serment de Platées', 'Uranie (Hérodote)', "Bataille de l'Artémision", "Bataille de l'Eurymédon", 'Bataille de Ladé (494 av. J.-C.)', 'Bataille de Marathon', 'Bataille du cap Mycale', 'Bataille de Platées', 'Bataille de Salamine', "Siège d'Érétrie", 'Bataille des Thermopyles', 'Bataille des Thermopyles', "300 : La Naissance d'un empire", '300 (bande dessinée)', '300 (film)', 'La Bataille des Thermopyles', 'Éphialtès de Trachis', 'Léonidas aux Thermopyles', 'Molon labe', 'Les Murailles de feu', 'Rue des Thermopyles', 'Spartatouille', 'Thermopyles', 'Abrocomès', 'Achéménès (satrape)', 'Adimante de Corinthe', 'Ariabignès', 'Ariphron', "Artaban (fils d'Hystaspe)", "Artaban l'Hyrcanien", 'Artapanus', 'Artapherne', "Artapherne (fils d'Artapherne)", 'Callimaque le Po

In [70]:
#################################################################
#### Scrapp the page of the given search  to give categories #####
#################################################################
def get_categories(name):
    url = "https://fr.wikipedia.org/wiki/" + name.replace(" ", "_")
    #requete get on the main url page
    page = requests.get(url)
    #verification of http code
    if page.status_code == 200:
        #parsing the page
        soup = BeautifulSoup(page.content, 'html.parser')
        #recuperation of the categories
        categories = soup.find_all('div', class_="mw-normal-catlinks")[0]
    else:
        #print an error if the page is unfindable
        print("Erreur lors de la requete")

    #creation of a empty list
    liste = []
    #recuperation of the <a> tag
    links = categories.find_all('a')
    #adding the content of the <a> tags to the list
    for link in links:
        liste.append(link.text)
    liste.remove("Catégories")
    return(liste)
get_categories(pages[0])

["Joueur d'esport coréen",
 "Joueur professionnel d'Overwatch",
 'Personnalité féminine sud-coréenne',
 'Personnalité utilisant un pseudonyme',
 'Naissance en juin 1999',
 'Naissance à Daejeon']

In [72]:
#############################################################################
#### Scrapp the page of the given search to give the user who contribute #####
#############################################################################
def get_user_page(name):
    # URL but this time we replace the space by underbar, because of the url system
    url = 'https://fr.wikipedia.org/w/index.php?title='+name.replace(" ", "_")+'&action=history&offset=&limit=500'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Getting of all users who contribute to the page, the maximum of printable user contribution is 500 user
    users = soup.find_all('a', {'class':'mw-userlink'})

    list_user = []
    #adding the user to the list
    for user in users:
        if user.text not in list_user:
            list_user.append(user.text)
    return(list_user)
get_user_page(pages[0])

['NeoBot',
 'OrlodrimBot',
 'Goombiis',
 'FR',
 'Jérémy-Günther-Heinz Jähnick',
 'AwkwardChester',
 'Galdrad']

In [73]:
#####################################################################################################################################
#### Scrapp the page of the user and returning the db with the 5 pages that they contribute and how much they contribute to them #####
#####################################################################################################################################
def sumup(name):
    # Creating the db with panda
    data = {'User_Name':  [],
        'Five_most_used_Theme': [],
        'Category': []
        }
    # calling the function to get the name of the contributer
    list_user = get_user_page(name)
    # Modifing the url to get the user contribution
    liste_underscore = [x.replace(" ", "_") for x in list_user]
    liste_plus = [x.replace(" ", "+") for x in list_user]
    df_local = pd.DataFrame(data)
    
    #
    for i in range(len(list_user)):
        url = 'https://fr.wikipedia.org/w/index.php?title=Sp%C3%A9cial:Contributions/'+liste_underscore[i]+'&target='+liste_plus[i]+'&offset=&limit=500'
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        
        # We count the number of iteration in the user contribution page
        page_list = []
        counter = {}
        for link in soup.find_all('a', {'class': 'mw-contributions-title'}):
            if 'Discussion' not in link.text and 'Wikipédia' not in link.text:
                element = link.text
                if element in counter:
                    counter[element] += 1
                else:
                    counter[element] = 1
        # We organize the number of article and select the 5 most updated page
        sorted_counter = dict(sorted(counter.items(), key=lambda item: item[1], reverse=True))
        top_five = dict(list(sorted_counter.items())[:5])
        
        match = False
        for index, row in df_local.iterrows():
            if row['User_Name'] == list_user[i] and row['Five_most_used_Theme'] == top_five:
                df_local.at[index, 'Category'] += ', ' + name
                match = True
                break
        if not match:
            df_local = df_local.append({'User_Name': list_user[i], 'Five_most_used_Theme': top_five, 'Category': name}, ignore_index=True)
    
    return df_local

sumup(pages[0])

,User_Name,Five_most_used_Theme,Category
0,NeoBot,"{'Projet:Aide et accueil/Twitter/Tweets': 39, ...",Geguri
1,OrlodrimBot,"{'Utilisateur:Pharma/À faire': 7, 'Projet:Ciné...",Geguri
2,Goombiis,"{'Assassin's Creed': 63, 'Utilisateur:Goombiis...",Geguri
3,FR,"{'Les Pyramides bleues': 8, 'Exposition pan-am...",Geguri
4,Jérémy-Günther-Heinz Jähnick,"{'Julien Rémy': 14, 'Projet Apollo': 13, 'Raym...",Geguri
5,AwkwardChester,"{'Liste des châteaux en France': 28, 'Femmes d...",Geguri
6,Galdrad,"{'Modèle:Coronavirus/Graph/Corée du Sud': 48, ...",Geguri


In [77]:
def final_table(category_pages):
    # Initialization of the list to store the tables
    tables = []
    for i in range(len(category_pages)):
        # Add each table created by the sumup function to the table list
        tables.append(sumup(category_pages[i]))
    # Merge tables into a single table
    final_df = pd.concat(tables, axis=0, ignore_index=True)
    final_df['Five_most_used_Theme'] = final_df['Five_most_used_Theme'].astype(str)
    final_df = final_df.groupby(["User_Name",'Five_most_used_Theme']).agg({"Category": lambda x: ",".join(x)})
    final_df.reset_index(inplace=True)
    # Return of the final table
    return final_df
final_df = final_table(pages)

In [78]:
# Lets show the whole table
final_df

,User_Name,Five_most_used_Theme,Category
0,2001:18c0:b20:1f00:49e5:5f7b:4720:458a,{'XQc': 1},XQc
1,2001:18c0:b20:1f00:9028:fba2:d064:dba7,{'XQc': 1},XQc
2,2001:56b:bd90:a900:457d:ac85:e776:d6df,{'XQc': 1},XQc
3,23.233.244.247,{'XQc': 2},XQc
4,24.122.125.87,{'XQc': 1},XQc
5,24.48.121.244,{'XQc': 1},XQc
6,2a01:cb04:c1a:b800:f023:e7b4:34b:eed5,"{'Club Deportivo Castellón': 3, 'XQc': 1}",XQc
7,66.131.194.246,{'XQc': 1},XQc
8,70.52.5.31,{'XQc': 2},XQc
9,70.83.184.57,"{'XQc': 1, 'Éric Lapointe': 1}",XQc


In [79]:
# We are doing it for another page
name = input('Quel est la page a consulte ?')
pages = get_category(name)
final_df = final_table(pages)

Quel est la page a consulte ?Guerres médiques


In [90]:
print("Voici toutes les categories dans "+name+".")
print(pages)
print("Et elle sont aux nombre de " + str(len(pages)) + ".")

Voici toutes les categories dans Guerres médiques.
['Guerres médiques', 'Calliope (Hérodote)', 'Canal de Xerxès', "Destruction achéménide d'Athènes", 'Guerres de la Ligue de Délos', 'Mardonios', 'Paix de Callias', 'Les Perses', 'Première guerre médique', 'Seconde guerre médique', 'Serment de Platées', 'Uranie (Hérodote)', "Bataille de l'Artémision", "Bataille de l'Eurymédon", 'Bataille de Ladé (494 av. J.-C.)', 'Bataille de Marathon', 'Bataille du cap Mycale', 'Bataille de Platées', 'Bataille de Salamine', "Siège d'Érétrie", 'Bataille des Thermopyles', 'Bataille des Thermopyles', "300 : La Naissance d'un empire", '300 (bande dessinée)', '300 (film)', 'La Bataille des Thermopyles', 'Éphialtès de Trachis', 'Léonidas aux Thermopyles', 'Molon labe', 'Les Murailles de feu', 'Rue des Thermopyles', 'Spartatouille', 'Thermopyles', 'Abrocomès', 'Achéménès (satrape)', 'Adimante de Corinthe', 'Ariabignès', 'Ariphron', "Artaban (fils d'Hystaspe)", "Artaban l'Hyrcanien", 'Artapanus', 'Artapherne', 

In [91]:
# Save the database in a csv file
final_df.to_csv('Guerres_mediques.csv', index=False)

In [84]:
#printing the whole table
final_df

,User_Name,Five_most_used_Theme,Category
0,*SM*,"{'Philippe Martinez': 4, ""Taux d'intérêt négat...","Bataille de l'Artémision,Léonidas Ier de Sparte"
1,-- Hizin --,"{'Brandon Teena': 7, 'Peine de mort en Arabie ...",Xerxès Ier
2,-=El Pingu=-,"{'Utilisateur:-=El Pingu=-': 28, 'Ramases': 8,...",Xerxès Ier
3,.Anja.,"{'Utilisatrice:.Anja./Obésité féline': 52, 'Li...",Les Perses
4,.anacondabot,"{'(762) Pulcova': 2, ""Capture d'écran"": 1, 'My...","Les Perses,Darius Ier,Hippias (tyran)"
...,...,...,...
2392,זבולון א,"{'Lycée français de Jérusalem': 6, 'Richelieu,...",Léonidas aux Thermopyles
2393,کولاک32,"{'Mausolée de Cyrus': 2, 'Xerxès Ier': 1}",Xerxès Ier
2394,प्रजापति,"{'Edmond Debeaumarché': 25, 'Utilisateur:प्रजा...",Léonidas Ier de Sparte
2395,ᖷᖴ,"{'Utilisateur:ᖷᖴ/common.js': 11, 'Timycha': 11...","Guerres médiques,Calliope (Hérodote),Mardonios..."
